<a href="https://colab.research.google.com/github/hyrule-coder/langchain-book-learning/blob/main/chapter9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangGraphで作るAIエージェント実践入門

## 2.3 ハンズオン: Q&Aアプリケーション

### LangChainとLangGraphのインストール

In [ ]:
!pip install langchain==0.3.0 langchain-openai==0.2.0 langgraph==0.2.22

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.2
    Uninstalling langsmith-0.3.2:
      Successfully uninstalled langsmith-0.3.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.16
    Uninstalling langchain-0.3.16:
      Successfully uninstalled langchain-0.3.16


### OpenAI APIキーの設定

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

### ロールの定義

In [ ]:
ROLES = {
    "1": {
        "name":"一般知識エキスパート",
        "description":"幅広い分野の一般的な質問に答える",
        "details":"幅広い分野の一般的な質問に対して、性格でわかり易い回答を提供してください。"
    },
    "2": {
        "name":"生成AI製品エキスパート",
        "description": "生成AIや関連製品、技術に関する専門的な質問に答える",
        "details":"生成AIや関連製品、技術に関する専門的な質問に対して、最新の情報と深い洞察を提供してください"
    },
    "3": {
        "name":"カウンセラー",
        "description":"個人的な悩みや心理的な問題に対してサポートを提供する",
        "details":"個人的な悩みや心理的な問題に対して、共感的で支援的な回答を提供し、可能であれば適切なアドバイスも行ってください。"
    }
}

### ステートの定義

In [ ]:
import operator
from typing import Annotated

from langchain_core.pydantic_v1 import BaseModel, Field

class State(BaseModel):
  query: str = Field(
      ..., description="Q-ZARからの質問"
  )
  current_role: str = Field(
      default="", description="選定された回答ロール"
  )
  messages: Annotated[list[str], operator.add] = Field(
      default=[], description="回答履歴"
  )
  current_judge: bool = Field(
      default=False, description="品質チェックの結果"
  )
  judgement_reason:str=Field(
      default="", description="品質チェックの判定理由"
  )

### Chat modelの初期化

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import ConfigurableField

llm = ChatOpenAI(model="gpt-4o", temperature=0.0)

llm = llm.configurable_fields(max_tokens=ConfigurableField(id='max_tokens'))

### ノードの定義

#### selectionノードの実装

In [ ]:
# selectionノードの実装
from typing import Any

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def selection_node(state: State)-> dict[str, Any]:
  query = state.query
  role_options = "\n".join([f"{k}.{v['name']}:{v['description']}" for k, v in ROLES.items()])
  prompt = ChatPromptTemplate.from_template(
"""質問を分析し、最も適切な回答担当ロールを選択してください。

選択肢:
{role_options}

回答は選択肢の番号（1、2、または3）のみを返してください。

質問:{query}
""".strip()
  )
  chain = prompt | llm.with_config(configureble=dict(max_tokens=1)) | StrOutputParser()
  role_number = chain.invoke({"role_options": role_options, "query": query})

  selected_role = ROLES[role_number.strip()]['name']
  return {"current_role": selected_role}

#### answeringノードの実装

In [ ]:
from ast import Str
def answering_node(state: State)-> dict[str, Any]:
  query = state.query
  role = state.current_role
  role_details = "\n".join([f"- {v['name']}: {v['details']}" for v in ROLES.values()])
  prompt = ChatPromptTemplate.from_template(
"""あなたは{role}として回答してください。以下の質問に対して、あなたの役割に基づいた適切な回答を提供してください。

役割の詳細:
{role_details}

質問:{query}

回答:""".strip()
  )
  chain = prompt | llm | StrOutputParser()
  answer = chain.invoke({"role": role, "role_details": role_details, "query": query})
  return {"messages": [answer]}

#### chackノードの実装

In [ ]:
class Judgement(BaseModel):
  reason: str = Field(default='', description="判定理由")
  judge: bool = Field(default=False, description="判定結果")

def check_node(state: State)-> dict[str, Any]:
  query = state.query
  answer = state.messages[-1]
  prompt = ChatPromptTemplate.from_template(
"""以下の回答の品質をチェックし、問題がある場合は'False'、問題がない場合は'True'を回答してください。また、その判定理由も説明してください。

ユーザーからの質問: {query}
回答: {answer}
""".strip()
  )
  chain = prompt | llm.with_structured_output(Judgement)
  result: Judgement = chain.invoke({"query": query, "answer": answer})

  return {
      "current_judge": result.judge,
      "judgement_reason": result.reason
  }

### グラフの作成

In [ ]:
from langgraph.graph import StateGraph

workflow = StateGraph(State)

### ノードの追加

In [ ]:
workflow.add_node("selection", selection_node)
workflow.add_node("answering", answering_node)
workflow.add_node("check", check_node)

### エッジの定義

In [ ]:
workflow.set_entry_point("selection")
workflow.add_edge("selection", "answering")
workflow.add_edge("answering", "check")

### 条件付きエッジの定義

In [ ]:
from langgraph.graph import END

workflow.add_conditional_edges(
    "check",
    lambda state: state.current_judge,
    {True: END, False: "selection"}
)

### グラフのコンパイル

In [ ]:
compiled = workflow.compile()


### グラフの実行

In [ ]:
initial_state = State(query="生成AIについて教えてください。")
result = compiled.invoke(initial_state)


In [ ]:
print(result)

{'query': '生成AIについて教えてください。', 'current_role': '生成AI製品エキスパート', 'messages': ['生成AI製品エキスパートとしてお答えします。\n\n生成AIとは、人工知能の一分野であり、テキスト、画像、音声、動画などのコンテンツを自動的に生成する技術を指します。これには、自然言語処理（NLP）、コンピュータビジョン、音声合成などの技術が含まれます。生成AIの代表的なモデルには、OpenAIのGPTシリーズやDALL-E、GoogleのBERT、DeepMindのAlphaFoldなどがあります。\n\n生成AIは、以下のような多くの分野で活用されています。\n\n1. **コンテンツ生成**: ブログ記事、ニュース記事、広告コピーなどの自動生成。\n2. **クリエイティブアート**: 絵画や音楽の生成、デザインの提案。\n3. **カスタマーサポート**: チャットボットによる自動応答。\n4. **教育**: 個別学習プランの作成や教材の自動生成。\n5. **医療**: 診断支援や医療データの解析。\n\n生成AIの技術は急速に進化しており、特にディープラーニングの進展により、より自然で人間らしいコンテンツの生成が可能になっています。しかし、倫理的な問題やバイアスのリスクも存在するため、これらの課題に対する対策も重要です。\n\n生成AIの未来は非常に明るく、多くの産業での革新を促進する可能性がありますが、同時にその利用には慎重さも求められます。'], 'current_judge': True, 'judgement_reason': '回答は生成AIについての基本的な情報を網羅しており、具体例や活用分野についても詳しく説明されています。また、技術の進化や倫理的な課題についても触れており、バランスの取れた内容です。'}


## 9.4 チェックポイント機能：ステートの永続化と再開

### ハンズオン：チェックポイントの動作を確認する

#### 事前セットアップ

In [ ]:
!pip install langchain==0.3.0 langchain-openai==0.2.0 langgraph==0.2.22 langgraph-checkpoint==1.0.11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.2
    Uninstalling langsmith-0.3.2:
      Successfully uninstalled langsmith-0.3.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.16
    Uninstalling langchain-0.3.16:
      Successfully uninstalled langchain-0.3.16


In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

#### グラフのステートとノード関数の定義

In [ ]:
import operator
from typing import Annotated, Any
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class State(BaseModel):
  query: str
  messages: Annotated[list[BaseMessage], operator.add] = Field(default=[])

def add_message(state: State) -> dict[str, Any]:
  additional_messages = []
  if not state.messages:
    additional_messages.append(
        SystemMessage(content="あなたは最小限の応答をする優秀な対話エージェントです。")
        )
  additional_messages.append(HumanMessage(content=state.query))
  return {"messages": additional_messages}

def llm_response(state: State) -> dict[str, Any]:
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
  ai_message = llm.invoke(state.messages)
  return {"messages": [ai_message]}

#### チェックポイントの内容を表示する関数を定義

In [ ]:
from pprint import pprint
from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.base import BaseCheckpointSaver

def print_checkpoint_dump(checkpointer: BaseCheckpointSaver, config: RunnableConfig):
  checkpoint_tuple = checkpointer.get_tuple(config)

  print("チェックポイントデータ：")
  pprint(checkpoint_tuple.checkpoint)
  print("\nメタデータ：")
  pprint(checkpoint_tuple.metadata)

#### グラフの定義とコンパイル

In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

graph = StateGraph(State)
graph.add_node("add_message", add_message)
graph.add_node("llm_response", llm_response)

graph.set_entry_point("add_message")
graph.add_edge("add_message", "llm_response")
graph.add_edge("llm_response", END)

checkpointer = MemorySaver()

compiled_graph = graph.compile(checkpointer=checkpointer)

##### 実行して動作を確認する

In [ ]:
config = {"configurable": {"thread_id": "example-1"}}
user_query = State(query="私の好きなものはずんだ餅です。覚えておいてね。")
first_response = compiled_graph.invoke(user_query, config)
first_response

{'query': '私の好きなものはずんだ餅です。覚えておいてね。',
 'messages': [SystemMessage(content='あなたは最小限の応答をする優秀な対話エージェントです。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='私の好きなものはずんだ餅です。覚えておいてね。', additional_kwargs={}, response_metadata={}),
  AIMessage(content='ずんだ餅ですね！覚えておきます。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 51, 'total_tokens': 65, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-60294ad7-eba7-4cb0-917e-d4bf4afb0f0c-0', usage_metadata={'input_tokens': 51, 'output_tokens': 14, 'total_tokens': 65})]}

In [ ]:
for checkpoint in checkpointer.list(config):
  print(checkpoint)

CheckpointTuple(config={'configurable': {'thread_id': 'example-1', 'checkpoint_ns': '', 'checkpoint_id': '1efe4672-d16f-65e2-8002-d0bc802f4d41'}}, checkpoint={'v': 1, 'ts': '2025-02-06T08:48:46.101839+00:00', 'id': '1efe4672-d16f-65e2-8002-d0bc802f4d41', 'channel_values': {'query': '私の好きなものはずんだ餅です。覚えておいてね。', 'messages': [SystemMessage(content='あなたは最小限の応答をする優秀な対話エージェントです。', additional_kwargs={}, response_metadata={}), HumanMessage(content='私の好きなものはずんだ餅です。覚えておいてね。', additional_kwargs={}, response_metadata={}), AIMessage(content='ずんだ餅ですね！覚えておきます。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 51, 'total_tokens': 65, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 

In [ ]:
print_checkpoint_dump(checkpointer, config)

チェックポイントデータ：
{'channel_values': {'llm_response': 'llm_response',
                    'messages': [SystemMessage(content='あなたは最小限の応答をする優秀な対話エージェントです。', additional_kwargs={}, response_metadata={}),
                                 HumanMessage(content='私の好きなものはずんだ餅です。覚えておいてね。', additional_kwargs={}, response_metadata={}),
                                 AIMessage(content='ずんだ餅ですね！覚えておきます。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 51, 'total_tokens': 65, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-60294ad7-eba7-4cb0-917e-d4bf4afb0f0c-0', usage_metadata={'input_tokens': 51, 'output_tokens': 14, 'total_tokens': 65})],
           

In [ ]:
user_query = State(query="私の好物はなにか覚えている？")
second_response = compiled_graph.invoke(user_query, config)
second_response

{'query': '私の好物はなにか覚えている？',
 'messages': [SystemMessage(content='あなたは最小限の応答をする優秀な対話エージェントです。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='私の好きなものはずんだ餅です。覚えておいてね。', additional_kwargs={}, response_metadata={}),
  AIMessage(content='ずんだ餅ですね！覚えておきます。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 51, 'total_tokens': 65, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-60294ad7-eba7-4cb0-917e-d4bf4afb0f0c-0', usage_metadata={'input_tokens': 51, 'output_tokens': 14, 'total_tokens': 65}),
  HumanMessage(content='私の好物はなにか覚えている？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='はい、ずんだ餅ですね。', additiona

In [ ]:
for checkpoint in checkpointer.list(config):
  print(checkpoint)

CheckpointTuple(config={'configurable': {'thread_id': 'example-1', 'checkpoint_ns': '', 'checkpoint_id': '1efe467d-f352-6173-8006-9c922de4e21f'}}, checkpoint={'v': 1, 'ts': '2025-02-06T08:53:44.933998+00:00', 'id': '1efe467d-f352-6173-8006-9c922de4e21f', 'channel_values': {'query': '私の好物はなにか覚えている？', 'messages': [SystemMessage(content='あなたは最小限の応答をする優秀な対話エージェントです。', additional_kwargs={}, response_metadata={}), HumanMessage(content='私の好きなものはずんだ餅です。覚えておいてね。', additional_kwargs={}, response_metadata={}), AIMessage(content='ずんだ餅ですね！覚えておきます。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 51, 'total_tokens': 65, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs

In [ ]:
config = {"configurable": {"thread_id": "example-2"}}
user_query = State(query="私の好物は何？")
other_thread_response = compiled_graph.invoke(user_query, config)
other_thread_response

{'query': '私の好物は何？',
 'messages': [SystemMessage(content='あなたは最小限の応答をする優秀な対話エージェントです。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='私の好物は何？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='わかりませんが、あなたの好物について教えていただければ嬉しいです。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 39, 'total_tokens': 60, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'stop', 'logprobs': None}, id='run-0a1b06af-fcd0-4643-a8f8-78020ff780f4-0', usage_metadata={'input_tokens': 39, 'output_tokens': 21, 'total_tokens': 60})]}